# Deploy Model to online endpoint with CLI V2

## In this notebook we will:

- Create the 3 scripts required for Model Deployment through the CLI V2 SDK

In [1]:
import os

# Create a folder for the experiment files
script_folder = 'CLI_V2_ManagedOnlineEndpoint'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

CLI_V2_ManagedOnlineEndpoint folder created


In [2]:
%%writefile ./CLI_V2_ManagedOnlineEndpoint/score.py

import os 
import json
import joblib
from pandas import json_normalize
import pandas as pd
import logging

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model/model.pkl')
    model = joblib.load(model_path)
    logging.info("Init complete")

# Called when a request is received
def run(raw_data):
    dict= json.loads(raw_data)
    df = json_normalize(dict['raw_data']) 
    y_pred = model.predict(df)
    print(type(y_pred))
    
    result = {"result": y_pred.tolist()}
    return result

Writing ./CLI_V2_ManagedOnlineEndpoint/score.py


In [3]:
%%writefile ./CLI_V2_ManagedOnlineEndpoint/endpoint.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: titanic-online-endpoint
description: "CLI V2 titanic online endpoint for mlflow model"
auth_mode: key
tags : {"CLIV2": "titanic"}

Writing ./CLI_V2_ManagedOnlineEndpoint/endpoint.yml


In [4]:
%%writefile ./CLI_V2_ManagedOnlineEndpoint/deployment.yml

$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: blue
endpoint_name: titanic-online-endpoint
model: azureml:chapter6_titanic_model@latest
code_configuration:
  code: .
  scoring_script: score.py
environment: azureml:env_for_sdkv2deploy@latest
instance_type: Standard_F2s_v2
instance_count: 1

Writing ./CLI_V2_ManagedOnlineEndpoint/deployment.yml
